# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [1]:
! pip install -U langchain

In [1]:
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "6a7ba2bcb3b543a339fd1bb6cdb25d46c29d532d65943ae6ea45184433cdd3a5"

In [8]:
from langchain_openai import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0,base_url="https://api.xiaoai.plus/v1",api_key="sk-tdqfro61NniG2LZd5394E5F28c51419d8b4dE7Db5a6105C4")

In [9]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [10]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [11]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.invoke(
    "成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output:  No. The answer to the first question is in the original question, and the answer to the second question is unrelated to the first. Therefore, follow-up questions are not needed. 

In [ ]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.invoke(
    "2023年大运会举办地在哪里？成都举办的大运会是第几届大运会？"
)

In [8]:
# Reason-only 答案（错误）
print(llm.invoke("成都举办的大运会是第几届大运会？"))



成都举办的大运会是第31届大运会。


In [9]:
# Reason-only 答案（错误）
print(llm.invoke("2023年大运会举办地在哪里？"))



2023年大运会的举办地将在中国的重庆市。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [16]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

In [17]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [18]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.invoke(
     "成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.

Follow up: 成都举办的大运会是哪一年？
Intermediate answer: 大运会所有必选竞赛项目全部是奥运项目，竞赛管理由国际单项体育组织按照奥运标准实施，是规模仅次于奥运会的世界综合性运动会，素有“小奥运会”之称。 2023年7月28日至8月8日，第31届世界大学生夏季运动会在四川成都举办。
Intermediate answer: 2023年7月28日至8月8日，第31届世界大学生夏季运动会在四川成都举办。

So the final answer is: 第31届

Follow up: 2023年大运会举办地在哪里？
Intermediate answer: 2018年3月5日正式宣布普莱西德湖成为2023年冬季大运会的主办城市，国际大体联官员称这是大运会申办历史上最强有力的一次。 由于2021年冬季世界大学生运动会因2019冠状病毒病疫情而取消，将大运会会旗正式移交给普莱西德湖的仪式改在2022年1月于都灵举行。
Intermediate answer: 2023年冬季大运会的举办地是普莱西德湖。

So the final answer is: 普莱西德湖

> Finished chain.


{'input': '成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？', 'output': '普莱西德湖'}